#### Scape blizzard API for spec/class ids, push into MDB

In [70]:
#import requests
import importlib
import blizzard_api
#import mysql
import mplusdb
import blizzard_credentials
#import pandas as pd
#import utils
#import time
#from concurrent.futures import ThreadPoolExecutor, as_completed
#import matplotlib.pyplot as plt
#import matplotlib.ticker as ticker
#import datetime

In [71]:
auth = blizzard_credentials.Credentials('.api_tokens')
access_token = auth.access_token

In [72]:
importlib.reload(blizzard_api)
caller = blizzard_api.Caller(access_token)
class_spec_table = caller.get_class_spec_table()

In [73]:
class_spec_table

[['mage', 8, 'arcane', 62, 'damage'],
 ['mage', 8, 'fire', 63, 'damage'],
 ['mage', 8, 'frost', 64, 'damage'],
 ['paladin', 2, 'holy', 65, 'healer'],
 ['paladin', 2, 'protection', 66, 'tank'],
 ['paladin', 2, 'retribution', 70, 'damage'],
 ['warrior', 1, 'arms', 71, 'damage'],
 ['warrior', 1, 'fury', 72, 'damage'],
 ['warrior', 1, 'protection', 73, 'tank'],
 ['druid', 11, 'balance', 102, 'damage'],
 ['druid', 11, 'feral', 103, 'damage'],
 ['druid', 11, 'guardian', 104, 'tank'],
 ['druid', 11, 'restoration', 105, 'healer'],
 ['death knight', 6, 'blood', 250, 'tank'],
 ['death knight', 6, 'frost', 251, 'damage'],
 ['death knight', 6, 'unholy', 252, 'damage'],
 ['hunter', 3, 'beast mastery', 253, 'damage'],
 ['hunter', 3, 'marksmanship', 254, 'damage'],
 ['hunter', 3, 'survival', 255, 'damage'],
 ['priest', 5, 'discipline', 256, 'healer'],
 ['priest', 5, 'holy', 257, 'healer'],
 ['priest', 5, 'shadow', 258, 'damage'],
 ['rogue', 4, 'assassination', 259, 'damage'],
 ['rogue', 4, 'outlaw', 

In [74]:
#tokenize into single string

tokenized = []
for spec in class_spec_table:
    class_name = spec[0].replace(' ', '_')
    spec_name = spec[2].replace(' ', '_')
    token = '%s_%s' % (class_name, spec_name)
    tokenized.append([spec[0], spec[1], spec[2], spec[3], spec[4], token]) #extend behaves weirdly

In [87]:
for i in tokenized:
    print('\'%s\',' % i[-1])

'mage_arcane',
'mage_fire',
'mage_frost',
'paladin_holy',
'paladin_protection',
'paladin_retribution',
'warrior_arms',
'warrior_fury',
'warrior_protection',
'druid_balance',
'druid_feral',
'druid_guardian',
'druid_restoration',
'death_knight_blood',
'death_knight_frost',
'death_knight_unholy',
'hunter_beast_mastery',
'hunter_marksmanship',
'hunter_survival',
'priest_discipline',
'priest_holy',
'priest_shadow',
'rogue_assassination',
'rogue_outlaw',
'rogue_subtlety',
'shaman_elemental',
'shaman_enhancement',
'shaman_restoration',
'warlock_affliction',
'warlock_demonology',
'warlock_destruction',
'monk_brewmaster',
'monk_windwalker',
'monk_mistweaver',
'demon_hunter_havoc',
'demon_hunter_vengeance',


#### Insert into db

In [76]:
importlib.reload(mplusdb)

mdb = mplusdb.MplusDatabase('.db_config')

query = ('INSERT IGNORE INTO spec '
         '(class_name, class_id, spec_name, spec_id, spec_role, token) '
         'VALUES (%s,%s,%s,%s,%s,%s)'
         )
mdb.raw_batch_insert(query = query, data = tokenized)

---

In [69]:
####
#
# Bonus track: write code to write code to generate one-hot encoded table for spec comps
#
# This needs to be in its own nb.


sql_script = ['use keyruns;\n'
              'CREATE table run_composition(\n',
              '\trun_id bigint unsigned not null PRIMARY KEY,\n']
for index, spec in enumerate(tokenized):
    token = spec[-1]
    if index < len(tokenized) - 1:
        sql_script.append('\t%s tinyint unsigned not null DEFAULT 0,\n' % token)
    else:
        sql_script.append('\t%s tinyint unsigned not null DEFAULT 0);\n' % token)        
print(sql_script)


with open('sql_scripts/test2.sql', 'w') as file:
    file.write("/* Generate one-hot encoded run comp table. Auto-generated.*/\n")
    for line in sql_script:
        file.write(line)
    file.write("/* to run: > mysql -u username -p < path-to\scriptName.sql */")

['use keyruns;\nCREATE table run_composition(\n', '\trun_id bigint unsigned not null PRIMARY KEY,\n', '\tmage_arcane tinyint unsigned not null DEFAULT 0,\n', '\tmage_fire tinyint unsigned not null DEFAULT 0,\n', '\tmage_frost tinyint unsigned not null DEFAULT 0,\n', '\tpaladin_holy tinyint unsigned not null DEFAULT 0,\n', '\tpaladin_protection tinyint unsigned not null DEFAULT 0,\n', '\tpaladin_retribution tinyint unsigned not null DEFAULT 0,\n', '\twarrior_arms tinyint unsigned not null DEFAULT 0,\n', '\twarrior_fury tinyint unsigned not null DEFAULT 0,\n', '\twarrior_protection tinyint unsigned not null DEFAULT 0,\n', '\tdruid_balance tinyint unsigned not null DEFAULT 0,\n', '\tdruid_feral tinyint unsigned not null DEFAULT 0,\n', '\tdruid_guardian tinyint unsigned not null DEFAULT 0,\n', '\tdruid_restoration tinyint unsigned not null DEFAULT 0,\n', '\tdeath_knight_blood tinyint unsigned not null DEFAULT 0,\n', '\tdeath_knight_frost tinyint unsigned not null DEFAULT 0,\n', '\tdeath_kn